In [1]:
!ls ./train

Separate	     dehaze_images	   dry-run-1	       results
clean_images	     dehaze_images_labels  haze_images
clean_images_labels  dry-run		   haze_images_labels


In [2]:
from PIL import Image
import os,sys

# PATH = "/home/pz2272/train/"
# OUTPUT_PATH = "/home/pz2272/yolo_datasets/CVPR_dehaze/"
PATH = "./train/"
OUTPUT_PATH = "./yolo_datasets/CVPR_dehaze/"
assert os.path.exists(PATH)

images = []
labels = []

# imgs_dir = os.listdir(PATH+"clean_images")
# labels_dir = os.listdir(PATH+"clean_images_labels")
imgs_dir = os.listdir(PATH+"dehaze_images")
labels_dir = os.listdir(PATH+"dehaze_images_labels")
imgs_dir.sort()
labels_dir.sort()

# imgs = [(os.path.join(PATH,"clean_images",img),img) for img in imgs_dir]
# labels = [(os.path.join(PATH,"clean_images_labels",label),label) for label in labels_dir]

imgs = [(os.path.join(PATH,"dehaze_images",img),img) for img in imgs_dir]
labels = [(os.path.join(PATH,"dehaze_images_labels",label),label) for label in labels_dir]
combo = zip(imgs,labels)

cnt = 0 
n = len(imgs_dir)

for img,lab in combo:
    image = Image.open(img[0])
    image_name = img[1]
    image_w, image_h = image.size
    label_name = lab[1]
    with open(lab[0],"r") as f:
        content = f.read().split("\n")
        output = ""
        for item in content[:-1]:
            temp = item.split(" ")
            #cls = temp[0]
            x_min = int(temp[1])
            y_min = int(temp[2])
            x_max = int(temp[3])
            y_max = int(temp[4])
            
            center_x = (x_min + x_max) / 2 
            center_y = (y_min + y_max) / 2
            width    = (x_max - x_min)
            height   = (y_max - y_min)
            
            center_x /= image_w 
            center_y /= image_h 
            width    /= image_w 
            height   /= image_h 
            
            output+= "{} {:.3f} {:.3f} {:.3f} {:.3f}\n".format(0, center_x, center_y, width, height)
        
        if cnt<0.7*n:
            image.save(os.path.join(OUTPUT_PATH,"images","train",image_name))
            with open(os.path.join(OUTPUT_PATH,"labels","train",label_name),"w") as f:
                f.write(output)       
        else:
            image.save(os.path.join(OUTPUT_PATH,"images","val",image_name))
            with open(os.path.join(OUTPUT_PATH,"labels","val",label_name),"w") as f:
                f.write(output)       
    cnt += 1